When you host a project using `git clone` and deploy it on a server, updating the code on the server after making changes locally or in the GitHub repository requires some steps. Here's how you can handle it:

---

### Does the Server Update Automatically?
No, the server does not update automatically after you push changes to the GitHub repository. You need to pull the changes on the server manually or automate the update process.

---

### Options for Updating the Code on the Server

#### **Option 1: Manual Update**
1. **Log in to the Server:**
   ```bash
   ssh user@server_ip
   ```
2. **Navigate to the Project Directory:**
   ```bash
   cd /path/to/your/project
   ```
3. **Pull the Latest Changes:**
   ```bash
   git pull origin main
   ```
4. **Restart the Application (if needed):**
   If it's a web app, restart the web server or app process (e.g., `nginx`, `gunicorn`, `node`, etc.):
   ```bash
   systemctl restart your-app.service
   ```
   Or, if using `pm2`:
   ```bash
   pm2 restart app_name
   ```

---

#### **Option 2: Automate Updates with Webhooks**
You can use **GitHub Webhooks** to trigger updates automatically when code is pushed to the repository.

1. **Set Up a Webhook in GitHub:**
   - Go to your GitHub repository → **Settings** → **Webhooks**.
   - Add a new webhook with:
     - **Payload URL**: URL of your server (e.g., `http://yourserver.com/webhook`).
     - **Content type**: `application/json`.
   - Save the webhook.

2. **Create a Webhook Listener on the Server:**
   Install a listener on your server to handle GitHub Webhooks.

   Example using Python and Flask:
   ```python
   from flask import Flask, request
   import os

   app = Flask(__name__)

   @app.route('/webhook', methods=['POST'])
   def webhook():
       if request.method == 'POST':
           os.system('cd /path/to/your/project && git pull origin main && systemctl restart your-app.service')
           return 'Updated successfully', 200
   ```

3. **Host the Listener:**
   Use a process manager like `systemd` or `pm2` to ensure the webhook listener is always running.

---

#### **Option 3: Use GitHub Actions for Deployment**
Set up a CI/CD pipeline to automate deployment.

- Define a workflow to SSH into your server and pull the latest code.
- Example workflow step for deployment:
  ```yaml
  - name: Deploy to Server
    env:
      SSH_PRIVATE_KEY: ${{ secrets.SSH_PRIVATE_KEY }}
      SERVER_IP: ${{ secrets.SERVER_IP }}
    run: |
      ssh -o StrictHostKeyChecking=no -i $SSH_PRIVATE_KEY user@$SERVER_IP "cd /path/to/your/project && git pull origin main && systemctl restart your-app.service"
  ```

---

#### **Option 4: Use a Deployment Tool**
Use tools like:
- **Ansible**: Automate server updates and app restarts.
- **Capistrano**: Ruby-based deployment automation.
- **Fabric**: Python-based tool for running remote SSH commands.

---

### Summary
- **Manual Pull**: Log in and pull changes using `git pull`.
- **Webhooks**: Automate updates triggered by GitHub pushes.
- **GitHub Actions**: Fully automate CI/CD with deployment scripts.
- **Deployment Tools**: Use tools like Ansible or Fabric for automation.

Choose the method based on your project's complexity and the level of automation.